# STM32 flashing from rpi

* STM32 Bootloader
  * [AN2606](https://www.st.com/resource/en/application_note/cd00167594-stm32-microcontroller-system-memory-boot-mode-stmicroelectronics.pdf)
     * STM32F40xxx/41xxx (Table 53, page 109)
     * Bootloader enabled on 
        * USART1: RX=PA10, TX=PA9
        * USART3: RX=PB11, TX=PB10 or RX=PC11, TX=PC10
     * Adafruit STM32F405
        * RX/TX pins are PB11, PB10 (UART3)
        * others not available

* [MicroPython UART REPL](https://github.com/micropython/micropython/blob/master/ports/stm32/boards/NUCLEO_F411RE/mpconfigboard.h)
  * [MicroPython Forum](https://forum.micropython.org/viewtopic.php?f=6&t=7425)

* Serial flasher:
  * [Python `stm32loader` - doesn't work](https://pypi.org/project/stm32loader/)
  * [C `stm32flash`](https://sourceforge.net/p/stm32flash/wiki/Home/)
  * [Instructions (C)](https://siliconjunction.wordpress.com/2017/03/21/flashing-the-stm32f-board-using-a-raspberry-pi-3/)



## Install C flasher

In [1]:
%%bash

cd /tmp
git clone https://git.code.sf.net/p/stm32flash/code stm32flash-code
cd stm32flash-code
sudo make install

Cloning into 'stm32flash-code'...
sudo: unable to resolve host 2d1d51c: No address associated with hostname
cc -Wall -g   -c -o dev_table.o dev_table.c
cc -Wall -g   -c -o i2c.o i2c.c
cc -Wall -g   -c -o init.o init.c
cc -Wall -g   -c -o main.o main.c
cc -Wall -g   -c -o port.o port.c
cc -Wall -g   -c -o serial_common.o serial_common.c
cc -Wall -g   -c -o serial_platform.o serial_platform.c
cc -Wall -g   -c -o stm32.o stm32.c
cc -Wall -g   -c -o utils.o utils.c
cd parsers && make parsers.a
make[1]: Entering directory '/tmp/stm32flash-code/parsers'
cc -Wall -g   -c -o binary.o binary.c
cc -Wall -g   -c -o hex.o hex.c
ar rc parsers.a binary.o hex.o
make[1]: Leaving directory '/tmp/stm32flash-code/parsers'
cc  -o stm32flash dev_table.o i2c.o init.o main.o port.o serial_common.o serial_platform.o stm32.o utils.o parsers/parsers.a
install -d /usr/local/bin
install -m 755 stm32flash /usr/local/bin
install -d /usr/local/share/man/man1
install -m 644 stm32flash.1 /usr/local/share/man/man1


In [2]:
%%bash

# retrieve device info
stm32flash /dev/ttyAMA2

Failed to init device.
stm32flash 0.6

http://stm32flash.sourceforge.net/

Interface serial_posix: 57600 8E1



In [8]:
%%bash

# put device in DFU mode!

# retrieve device info
stm32flash /dev/ttyAMA2

# flash (be patient, the 2nd flash takes a couple of minutes)
stm32flash -v -S 0x08000000 -w $IOT/micropython/ports/stm32/build-MOTOR_HAT/firmware0.bin /dev/ttyAMA2
stm32flash -v -S 0x08020000 -w $IOT/micropython/ports/stm32/build-MOTOR_HAT/firmware1.bin /dev/ttyAMA2

stm32flash 0.6

http://stm32flash.sourceforge.net/

Interface serial_posix: 57600 8E1
Version      : 0x31
Option 1     : 0x00
Option 2     : 0x00
Device ID    : 0x0413 (STM32F40xxx/41xxx)
- RAM        : Up to 128KiB  (12288b reserved by bootloader)
- Flash      : Up to 1024KiB (size first sector: 1x16384)
- Option RAM : 16b
- System RAM : 30KiB

stm32flash 0.6

http://stm32flash.sourceforge.net/

Using Parser : Raw BINARY
Interface serial_posix: 57600 8E1
Version      : 0x31
Option 1     : 0x00
Option 2     : 0x00
Device ID    : 0x0413 (STM32F40xxx/41xxx)
- RAM        : Up to 128KiB  (12288b reserved by bootloader)
- Flash      : Up to 1024KiB (size first sector: 1x16384)
- Option RAM : 16b
- System RAM : 30KiB
Write to memory
Erasing memory
Wrote and verified address 0x08003974 (100.00%) Done.

stm32flash 0.6

http://stm32flash.sourceforge.net/

Using Parser : Raw BINARY
Interface serial_posix: 57600 8E1
Version      : 0x31
Option 1     : 0x00
Option 2     : 0x00
Device ID    : 0x0413

## Flash STM32 via /dev/serial (MOTOR_HAT)

In [2]:
%%bash

# enter boot mode

echo "21" > /sys/class/gpio/export
echo "27" > /sys/class/gpio/export
sleep 0.1
echo "out" > /sys/class/gpio/gpio21/direction
echo "out" > /sys/class/gpio/gpio27/direction
# boot mode
echo "1" >  /sys/class/gpio/gpio27/value
sleep 0.1
echo "0" >  /sys/class/gpio/gpio21/value
sleep 0.1
echo "1" >  /sys/class/gpio/gpio21/value
echo "21" > /sys/class/gpio/unexport
echo "27" > /sys/class/gpio/unexport
sleep 0.1

echo "STM32 in boot mode ..."

# retrieve device info
stm32flash /dev/ttyAMA2

# flash
stm32flash -v -S 0x08000000 -w $IOT/micropython/ports/stm32/build-MOTOR_HAT/firmware0.bin /dev/ttyAMA2
stm32flash -v -S 0x08020000 -w $IOT/micropython/ports/stm32/build-MOTOR_HAT/firmware1.bin /dev/ttyAMA2

# reset (no boot mode)

echo "21" > /sys/class/gpio/export
echo "27" > /sys/class/gpio/export
sleep 0.1
echo "out" > /sys/class/gpio/gpio21/direction
echo "out" > /sys/class/gpio/gpio27/direction
# normal mode
echo "0" >  /sys/class/gpio/gpio27/value
sleep 0.1
echo "0" >  /sys/class/gpio/gpio21/value
sleep 0.5
echo "1" >  /sys/class/gpio/gpio21/value
echo "21" > /sys/class/gpio/unexport
echo "27" > /sys/class/gpio/unexport

echo "STM32 reset."

STM32 in boot mode ...
stm32flash 0.6

http://stm32flash.sourceforge.net/

Interface serial_posix: 57600 8E1
Version      : 0x31
Option 1     : 0x00
Option 2     : 0x00
Device ID    : 0x0413 (STM32F40xxx/41xxx)
- RAM        : Up to 128KiB  (12288b reserved by bootloader)
- Flash      : Up to 1024KiB (size first sector: 1x16384)
- Option RAM : 16b
- System RAM : 30KiB

stm32flash 0.6

http://stm32flash.sourceforge.net/

Using Parser : Raw BINARY
Interface serial_posix: 57600 8E1
Version      : 0x31
Option 1     : 0x00
Option 2     : 0x00
Device ID    : 0x0413 (STM32F40xxx/41xxx)
- RAM        : Up to 128KiB  (12288b reserved by bootloader)
- Flash      : Up to 1024KiB (size first sector: 1x16384)
- Option RAM : 16b
- System RAM : 30KiB
Write to memory
Erasing memory
Wrote and verified address 0x08003974 (100.00%) Done.

stm32flash 0.6

http://stm32flash.sourceforge.net/

Using Parser : Raw BINARY
Interface serial_posix: 57600 8E1
Version      : 0x31
Option 1     : 0x00
Option 2     : 0x0

## Verify

In [3]:
%%host

import serial, time

with serial.Serial("/dev/ttyAMA1", baudrate=115200, timeout=1.0) as dev:
    dev.write(b"\rprint('hello', 2**16)\rprint(2**400)\r")
    while True:
        recv = dev.read().decode()
        if len(recv) == 0:
            break
        print(recv, end='')
        time.sleep(0.01)


>>> print('hello', 2**16)
hello 65536
>>> print(2**400)
2582249878086908589655919172003011874329705792829223512830659356540647622016841194629645353280137831435903171972747493376
>>> 

In [3]:
%connect 'serial:///dev/ttyAMA1'

Connected to 1c:00:26:00:09:50:52:42:4e:30:39:20 @ serial:///dev/ttyAMA1


In [3]:
print(2**300)

2037035976334486086268445688409378161051468393665936250636140449354381299763336706183397376


In [3]:
%rlist

                            flash/
    528  Dec 31 23:00 2014      README.txt
    366  Dec 31 23:00 2014      boot.py
     34  Dec 31 23:00 2014      main.py
   2999  Dec 31 23:00 2014      pybcdc.inf


In [3]:
%rdiff

DELETE  /flash/pybcdc.inf
DELETE  /flash/main.py
DELETE  /flash/boot.py
DELETE  /flash/README.txt
DELETE  /flash
DELETE  /


In [3]:
%discover

1c:00:26:00:09:50:52:42:4e:30:39:20  serial:///dev/ttyAMA1  
